In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor


In [2]:
df=pd.read_csv('/home/admin/Downloads/FSDSRegression-main/notebooks/data/gemstone.csv')

In [3]:
X = df.drop('price',axis=1)
y = df['price']

In [4]:
X

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [5]:
categorical_colums = X.select_dtypes(include='object').columns
numerical_colums = X.select_dtypes(exclude='object').columns

In [6]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [7]:
num_pipeline = Pipeline([
    ('impute',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

cat_pipeline = Pipeline([
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler(with_mean=False))
])

preprocessor = ColumnTransformer([
    ('num_pipe',num_pipeline,numerical_colums),
    ('cat_pipe',cat_pipeline,categorical_colums)
])

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [9]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [10]:
import pickle
import dill

In [11]:
with open('preprocessor.pkl', 'wb') as file_obj:
    dill.dump(file=file_obj, obj=preprocessor)

In [12]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [13]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [14]:
regression.coef_

array([-4.49143681e-02,  6.43365918e+03, -1.32758240e+02, -7.04288742e+01,
       -1.72031317e+03, -4.99289729e+02, -6.33921992e+01,  7.24455152e+01,
       -4.60416107e+02,  6.50764277e+02])

In [15]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [16]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1013.9048025658445
MAE: 674.0257224618157
R2 score 93.68908132630602


Lasso
Model Training Performance
RMSE: 1013.8784227049384
MAE: 675.0716918421083
R2 score 93.68940971806553


Ridge
Model Training Performance
RMSE: 1013.9060209095825
MAE: 674.0557936093446
R2 score 93.6890661594524


Elasticnet
Model Training Performance
RMSE: 1533.4194703386768
MAE: 1060.742542700526
R2 score 85.56488759855306


